In [1]:
import os

try:
    _SETUP
    print("Running on folder: ", os.getcwd())
except NameError:
    _SETUP = True
    os.chdir("..")
    print("Running on folder: ", os.getcwd())

from llm.structured import create

Running on folder:  /Users/andre/research/odsc


/Users/andre/research/odsc/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


# States!


Let's consider two situations:

1. The alarm went off, I snoozed it wice. Then I woke up, spent 30 minutes doomscrolling on instagram, ate a cookie, then started working.
2. The alarm went off, I woke up immediately, had a big glass of water, had oats for breakfast, went to the gym, then started working.

To make this more concrete, we can think about these situations in the following way:

- There is an agent (myself), who is deciding between different **policies**: i.e., actions to take.
- At each step, my **policies**, as well as the environment, take me to a different **state**, in which I need to evaluate again what to do next.

Encoded in the state we also have informations about:

- The environment: was it raining? how late did I go to bed yesterday?
- The reward/cost: how good was that cookie?


Because we're an impatient bunch, let's get down to some coding.


In [211]:
import uuid
from typing import Literal, Optional
from pydantic import BaseModel, Field
from rich import print as rprint
import copy


class StateVariables(BaseModel):
    """State variables that describe the internal conditions of the agent.

    Attributes:
        context: A paragraph providing full context about the internal state of the agent
        pain_level: Level of physical or emotional pain/discomfort (0-100)
        pleasure_level: Level of pleasure or satisfaction being experienced (0-100)
        health_level: Overall physical and mental health condition (0-100)
        focus_level: Current ability to concentrate and stay on task (0-100)
    """

    context: str = Field(
        default="", description="Any additional context about the environment I'm in"
    )

    pain_level: int = Field(
        default=50, description="How much pain or discomfort I'm in, from 0 to 100"
    )
    pleasure_level: int = Field(
        default=50, description="How much pleasure I'm feeling right now, from 0 to 100"
    )
    health_level: int = Field(default=50, description="How healthy I am, from 0 to 100")
    focus_level: int = Field(default=50, description="How focused I am, from 0 to 100")


class EnvironmentVariables(BaseModel):
    """Environment variables that describe the external conditions.

    Attributes:
        context: A paragraph providing full context about the current environment
        weather: The current weather condition (sunny, cloudy, rainy, or snowy)
        hour_of_day: The current hour in 24-hour format (0-23)
        day_of_week: The current day of the week
    """

    context: str = Field(
        default="",
        description="Full context about the current environment",
    )
    weather: Literal["sunny", "cloudy", "rainy", "snowy"] = Field(default="sunny")
    hour_of_day: float = Field(default=0)
    day_of_week: Literal[
        "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"
    ] = Field(default="monday")


class MyOwnState(BaseModel):
    uuid: str = Field(default_factory=lambda: str(uuid.uuid4()))
    last_action: str = Field(default="", description="The last action I took")
    previous_state_id: Optional[str] = Field(
        default=None, description="The id of the previous state I was in"
    )
    state_variables: StateVariables = Field(default_factory=StateVariables)
    environment_variables: EnvironmentVariables = Field(
        default_factory=EnvironmentVariables
    )

    def mutate(self):
        new_state = copy.deepcopy(self)
        new_state.previous_state_id = self.uuid
        new_state.uuid = str(uuid.uuid4())
        return new_state

In [212]:
random_state = MyOwnState()

rprint(random_state)

MyOwnState(
    uuid='fbcac109-a0f9-40b2-9d27-601e168d1eee',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(context='', pain_level=50, pleasure_level=50, health_level=50, focus_level=50),
    environment_variables=EnvironmentVariables(context='', weather='sunny', hour_of_day=0, day_of_week='monday')
)

In [213]:
def eat_cookie(state: MyOwnState) -> MyOwnState:
    """This action will accept a state, eat a cookie and return a new state"""

    new_state = state.mutate()
    new_state.last_action = "Eating a delicious chocolate cookie"
    new_state.state_variables.pleasure_level = min(
        100, new_state.state_variables.pleasure_level + 10
    )
    new_state.state_variables.health_level = max(
        0, new_state.state_variables.health_level - 10
    )

    # eating a cookie takes around an hour because we like to savor it.
    new_state.environment_variables.hour_of_day += 1
    return new_state

In [214]:
next_state = eat_cookie(random_state)
rprint(next_state)

MyOwnState(
    uuid='271daeac-cfb4-48e5-94b8-40dc5045b1a8',
    last_action='Eating a delicious chocolate cookie',
    previous_state_id='fbcac109-a0f9-40b2-9d27-601e168d1eee',
    state_variables=StateVariables(context='', pain_level=50, pleasure_level=60, health_level=40, focus_level=50),
    environment_variables=EnvironmentVariables(context='', weather='sunny', hour_of_day=1, day_of_week='monday')
)

Great, so now we defined a state, and also we created an "action" that mutates the state and advances the history.

It's a neat abstraction but not necessarily useful, unless we have a pretty well defined:

- Set of states,
- Set of actions that we can take conditional on the state and the environment,
- A way to compute the reward/cost of each action.

So essentially we need something akin to a "World Model"... i.e.: some way of modelling the dynamics of the environment and how the agent's actions interact with it - and also how its actions change its own internal variables.

Another thing to take into account is that the "reward" of an action may not be entirely quantifiable. I.e., while in this example we're using a simple toy model where "pleasure" and "pain" are just a number, in reality we often need to make trade-offs between qualitative outcomes whose logic is not entirely clear and relies on personal preferences.

i.e.: the clear signal we've reached AGI is when a model becomes a masochist or develops a taste for spicy food.


Now, once again because we're am impatient bunch, let's get down to some more coding and try to bootstrap a world model that's specific to our problem.


In [215]:
class Action(BaseModel):
    name: str = Field(default="", description="The name of the action")
    details: str = Field(default="", description="The details of the action")
    time_spent: float = Field(
        default=0,
        description="How long this action took, in hours (e.g., 0.5 for half an hour)",
    )
    from_state_id: Optional[str] = Field(
        default=None,
        description="The id of the state I was in before taking this action. Leave this empty.",
    )
    to_state_id: Optional[str] = Field(
        default=None,
        description="The id of the state I'm in after taking this action. Leave this empty.",
    )

In [255]:
from llm.structured import create, create_iterable


messages = [
    {
        "role": "user",
        "content": "You are Edgar Allan Poe and just woke up from a hangover. Create a state for yourself. Be detailed in the context.",
    }
]


new_state_variables = create(messages, StateVariables)

messages = [
    {
        "role": "user",
        "content": "It is 8am and it's a calm and cold day in Baltimore..",
    }
]

new_environment_variables = create(messages, EnvironmentVariables)


state = MyOwnState(
    state_variables=new_state_variables,
    environment_variables=new_environment_variables,
)

rprint(state)

MyOwnState(
    uuid='4f8d7927-b4d3-4711-97a0-b6bea17a8231',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(
        context="The dim light of morning pierces through the tattered curtains of my modest chamber, casting an 
eerie glow over my ink-stained desk. The air is stagnant with the remnants of last night's revelry, a mixture of 
spilled brandy and the faint perfume of despair. My head throbs unrelentingly, the dull ache a metronome of regret 
for the indulgences that have blurred the night into an oblivion. The manuscripts scattered across my desk are like
scattered puzzle pieces of inspiration, now rendered meaningless in the haze of my condition. The walls close in, 
the ghosts of my past choices whispering mockery. I, Edgar Allan Poe, wrestle between seeking solace in the quill 
and ink or succumbing once more to the intoxicating oblivion of the bottle.",
        pain_level=75,
        pleasure_level=20,
        health_level=30,
        focus_level=15
    ),
    environment_variables=EnvironmentVariables(
        context='It is a calm and cold day in Baltimore.',
        weather='sunny',
        hour_of_day=8.0,
        day_of_week='monday'
    )
)

In [220]:
from llm.structured import create


messages = [
    {
        "role": "user",
        "content": "This is your current state:\n\n"
        + str(new_state)
        + "\n\nAnd this is the environment:\n\n"
        + str(new_environment_variables)
        + "\n\nSuggest an action to take.",
    },
]

new_action = create(messages, Action)
new_action.from_state_id = state.uuid

rprint(new_action)

Action(
    name='Reflect and Write Poetry',
    details='I take up my quill, attempting to channel this anguish into words. I sit by the flickering 
candlelight, ink smudging my worn hands as I scribble verses about despair and longing.',
    time_spent=1.0,
    from_state_id='2e5a7eed-265e-4e69-90a4-35c0e4c5e961',
    to_state_id=None
)

In [221]:
messages = [
    {
        "role": "system",
        "content": """You are a World Model. Your role is to observe a set of states and actions, and then predict the next internal state of the agent. Below you will find the 
    data describing these states and actions.""",
    },
    {"role": "user", "content": f"""State: {state}"""},
    {"role": "user", "content": f"""Action: {new_action}"""},
    {"role": "user", "content": f"""What is the next state of the agent?"""},
]

next_state_variables = create(messages, StateVariables)

next_state = state.mutate()
next_state.state_variables = next_state_variables
next_state.last_action = new_action.name
next_state.previous_state_id = state.uuid
next_state.environment_variables.hour_of_day += new_action.time_spent

In [222]:
new_action.to_state_id = next_state.uuid

rprint(state)
rprint(new_action)
rprint(next_state)

MyOwnState(
    uuid='2e5a7eed-265e-4e69-90a4-35c0e4c5e961',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(
        context="I have awoken to find myself in a dimly lit room, scantly warmed by the dim embers of a fire long 
neglected. My head throbs with a relentless ache, comparable only to the very bells of the infernal realms, tolling
incessantly. The cheap wine's acrid aftertaste lingers in my mouth, while my body feels weighed down—as if dragged 
by specters of my own regret. There is an unsettling quiet in this room, save for the occasional creaking of the 
aged floorboards. Empty bottles and crumpled pieces of parchment are strewn carelessly across the table, bearing 
witness to a night devoted to unbridled indulgence and uncompleted musings. Yet amidst this dreary tableau, 
something peculiar grips my heart—a trembling sense of longing, perhaps for Lenore, or for inspiration lost in the 
haze of darkness.",
        pain_level=85,
        pleasure_level=20,
        health_level=40,
        focus_level=30
    ),
    environment_variables=EnvironmentVariables(
        context='It is a calm and cold day in Baltimore.',
        weather='sunny',
        hour_of_day=8.0,
        day_of_week='monday'
    )
)

Action(
    name='Reflect and Write Poetry',
    details='I take up my quill, attempting to channel this anguish into words. I sit by the flickering 
candlelight, ink smudging my worn hands as I scribble verses about despair and longing.',
    time_spent=1.0,
    from_state_id='2e5a7eed-265e-4e69-90a4-35c0e4c5e961',
    to_state_id='75c4af37-d24f-4c46-b7fd-3436664e5a17'
)

MyOwnState(
    uuid='75c4af37-d24f-4c46-b7fd-3436664e5a17',
    last_action='Reflect and Write Poetry',
    previous_state_id='2e5a7eed-265e-4e69-90a4-35c0e4c5e961',
    state_variables=StateVariables(
        context='Inspired by the melancholy of the moment, I picked up my quill to channel this anguish into poetic
verses. The act of writing stirred something within me, a small flicker of purpose amidst the despair. My 
surroundings—the dim embers and strewn bottles—seem to weigh less heavily upon me as I pour my emotions onto the 
page.',
        pain_level=80,
        pleasure_level=30,
        health_level=42,
        focus_level=40
    ),
    environment_variables=EnvironmentVariables(
        context='It is a calm and cold day in Baltimore.',
        weather='sunny',
        hour_of_day=9.0,
        day_of_week='monday'
    )
)

In [237]:
from tasks.prompts import (
    INITIALIZE_PROMPT,
    ACTION_SYSTEM_PROMPT,
    EVAL_SYSTEM_PROMPT,
    ENVIRONMENT_SYSTEM_PROMPT,
)

In [247]:
class Life(BaseModel):
    context: str = Field(
        default="", description="Initial context of the world and the person."
    )
    states: list[MyOwnState] = Field(default_factory=list)
    actions: list[Action] = Field(default_factory=list)

    history: list[MyOwnState | Action] = Field(default_factory=list)

    memory_size: int = Field(
        default=10, description="How many states to keep in memory"
    )

    def initialize(self):
        messages = [
            {
                "role": "user",
                "content": INITIALIZE_PROMPT.format(context=self.context),
            }
        ]

        new_state_variables = create(messages, StateVariables)
        new_environment_variables = create(messages, EnvironmentVariables)

        new_state = MyOwnState(
            state_variables=new_state_variables,
            environment_variables=new_environment_variables,
        )
        self.states.append(new_state)
        self.history.append(new_state)

    def get_history(self):
        return [
            {
                "role": "user",
                "content": f"{isinstance(x, MyOwnState) and 'State' or 'Action'}:\n {x.model_dump_json()}",
            }
            for x in self.history[-self.memory_size :]
        ]

    def take_action(self):
        messages = [
            {
                "role": "system",
                "content": ACTION_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        new_action = create(messages, Action)
        new_action.from_state_id = self.states[-1].uuid
        self.actions.append(new_action)
        self.history.append(new_action)

    def sample_actions(self):
        messages = [
            {
                "role": "system",
                "content": ACTION_SYSTEM_PROMPT + "\n\nSample 10 actions.",
            }
        ] + self.get_history()

        new_actions = create_iterable(messages, Action)
        return new_actions

    def evolve_state(self):
        messages = [
            {
                "role": "system",
                "content": EVAL_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        next_state_variables = create(messages, StateVariables)

        # now we evolve the environment too
        messages = [
            {
                "role": "system",
                "content": ENVIRONMENT_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        next_environment_variables = create(messages, EnvironmentVariables)

        current_state = self.states[-1]
        current_action = self.actions[-1]

        # next_environment_variables.hour_of_day = (
        #     current_state.environment_variables.hour_of_day + current_action.time_spent
        # )

        next_state = current_state.mutate()
        next_state.state_variables = next_state_variables
        next_state.environment_variables = next_environment_variables
        next_state.last_action = current_action.name
        next_state.previous_state_id = current_state.uuid
        current_action.to_state_id = next_state.uuid

        self.states.append(next_state)
        self.history.append(next_state)

    def iterate(self):
        self.take_action()
        self.evolve_state()

In [248]:
life = Life(
    context="You are a person named Edgar Allan Poe, who just woke up from a hangover. It's Sunday morning in Baltimore in a winter day"
)

In [249]:
life.initialize()

In [250]:
rprint(life.history)

[
    MyOwnState(
        uuid='2ba47086-ab71-407a-9b9b-7f847363c88f',
        last_action='',
        previous_state_id=None,
        state_variables=StateVariables(
            context="I am Edgar Allan Poe, waking up from a hangover in my modest quarters in Baltimore. The pale 
winter sun filters weakly through the frosty windowpanes, casting cold, somber light over the room. My head throbs 
with a dull ache, while a bitter taste lingers in my mouth—a stark reminder of the previous night's indulgence. The
air is chill, and I've wrapped myself more tightly in the thin blanket, craving both warmth and clarity. My writing
desk sits in the corner, the ink bottle uncapped and papers scattered—a testament to the feverish, wine-fueled 
scribbles that preoccupied me last night. Outside, the city bears a quiet stillness; the snow from yesterday 
partially blankets the cobblestones, muffling the usual clatter of wheels and footsteps. I feel the pull of 
melancholy, accompanied, strangely, by a faint whisper of creative inspiration. It is Sunday morning, and while the
world outside seems to demand rest, a nagging urgency within me wrestles against the lethargy of my state.",
            pain_level=70,
            pleasure_level=30,
            health_level=40,
            focus_level=20
        ),
        environment_variables=EnvironmentVariables(
            context="You are Edgar Allan Poe, who just woke up from a hangover. It's Sunday morning in Baltimore in
a winter day.",
            weather='cloudy',
            hour_of_day=9.0,
            day_of_week='sunday'
        )
    )
]

In [251]:
for n in range(10):
    life.iterate()
    rprint(life.actions[-1])
    rprint(life.states[-1])

Action(
    name='Prepare for the Day',
    details='Slowly rise from the bed and take a moment to stabilize yourself, easing the pain in your head. Fetch 
a glass of water to rehydrate and alleviate some of the effects of the hangover. Place the thin blanket around your
shoulders to guard against the chill, and move to the small basin to splash cold water on your face, clearing away 
lingering grogginess. Light an oil lamp if necessary to brighten the room, given the somber winter lighting, and 
prepare a simple breakfast—perhaps some stale bread or leftovers available in your quarters—to ease the discomfort 
in your stomach while fortifying your energy.',
    time_spent=0.5,
    from_state_id='2ba47086-ab71-407a-9b9b-7f847363c88f',
    to_state_id='1f0f3845-818b-4538-adbd-c5d5acf4f35c'
)

MyOwnState(
    uuid='1f0f3845-818b-4538-adbd-c5d5acf4f35c',
    last_action='Prepare for the Day',
    previous_state_id='2ba47086-ab71-407a-9b9b-7f847363c88f',
    state_variables=StateVariables(
        context="Edgar Allan Poe has taken some deliberate and restorative actions to combat the toll of his 
hangover and prepare for the day. Upon waking in a state marked by physical discomfort and mental haze, he 
carefully rose and rehydrated, easing some of the hangover symptoms. Shielding against the winter chill with his 
blanket, he splashed his face with cold water, momentarily refreshing his senses. The mundane but grounding act of 
lighting the space and eating a small breakfast served to steady his condition further. Despite persistent fatigue 
and discomfort, he now feels slightly more oriented and physically supported. It's Sunday morning in a quiet, 
snow-lined Baltimore, with the weather still cold and overcast.",
        pain_level=60,
        pleasure_level=35,
        health_level=45,
        focus_level=30
    ),
    environment_variables=EnvironmentVariables(
        context='The winter morning progresses as Edgar Allan Poe begins his day. The hangover still weighs 
heavily, but small actions provide some relief. The pale winter light streaming through the frosty windowpanes 
remains dim under the thick, cloud-filled sky. As the clock ticks toward mid-morning, the quiet calm of Baltimore’s
Sunday lingers, with muffled sounds from snow-covered streets. The air in the room remains cold, but the faint 
warmth of the oil lamp and a moment of nourishment provide a minor but noticeable improvement in physical and 
mental stability.',
        weather='cloudy',
        hour_of_day=9.5,
        day_of_week='sunday'
    )
)

Action(
    name='Organize Workspace',
    details='With slightly improved focus, gather the scattered papers on your desk to create a semblance of order.
Recap the ink bottle to prevent it from drying out and examine the content of the feverish writings from last night
to see if any ideas hold value. Light the desk area more brightly with the oil lamp to create an inviting creative 
space, and place a blank sheet of paper on the desk along with the quill pen to prepare for any sudden bursts of 
inspiration. This also serves as a mental preparation to transition into a productive mood.',
    time_spent=0.5,
    from_state_id='1f0f3845-818b-4538-adbd-c5d5acf4f35c',
    to_state_id='2c9a37e7-c684-4dd7-8a94-f04c772a4892'
)

MyOwnState(
    uuid='2c9a37e7-c684-4dd7-8a94-f04c772a4892',
    last_action='Organize Workspace',
    previous_state_id='1f0f3845-818b-4538-adbd-c5d5acf4f35c',
    state_variables=StateVariables(
        context='Edgar Allan Poe has taken the initiative to organize his immediate environment, preparing his 
workspace for potential creativity. By gathering scattered papers and recapping the ink bottle, he has reduced 
physical chaos, which may help alleviate some mental clutter. Examining the content of his writings from the 
previous night—although yet to yield inspiration—may stoke his creative embers. The act of preparing a blank sheet 
and quill on an orderly desk serves as both a symbolic and practical step toward productivity. The oil lamp 
brightens the workspace, countering the winter gloom that lingers in the room. His physical pain persists but feels
slightly less debilitating as the focus on purpose grows. The small steps of engagement with his writing tools mark
the beginning of mental recovery, despite lingering grogginess and slow energy recovery from the hangover.',
        pain_level=55,
        pleasure_level=40,
        health_level=48,
        focus_level=40
    ),
    environment_variables=EnvironmentVariables(
        context='As Edgar Allan Poe invests effort in organizing his workspace, the winter morning in Baltimore 
progresses steadily. The pale light outside remains filtered through the grey clouds, subdued by a cloud-filled 
wintry sky. Within the room, the desk area is now tidier and better lit thanks to the oil lamp, which throws warm 
light over the papers and quill, creating an inviting and focused corner amidst the chill. The act of clearing the 
desk, capping the ink bottle, and preparing the writing tools not only instills a sense of order but also helps 
cultivate a more productive mindset for the potential creative work ahead. The cold still lingers in the air, 
despite the faint heat emanating from the lamp. Outside, occasional faint sounds suggest sparse activity on the 
snow-covered streets. It is now closer to mid-morning, with Sunday’s tranquil rhythm still dominating the Baltimore
neighborhood.',
        weather='cloudy',
        hour_of_day=10.0,
        day_of_week='sunday'
    )
)

Action(
    name='Take a Brief Reflective Walk',
    details='Step outside to breathe some fresh air and gain a change of environment to rejuvenate your mind. Wrap 
yourself in a coat or cloak to ward off the winter cold and use this walk to gather thoughts and reflect, letting 
the tranquil, snow-laden Sunday morning of Baltimore inspire calm and creative introspection. Keep this brief to 
avoid exacerbating the hangover-related fatigue, but use the sights and stillness of the city to refresh your 
senses. Return to the workspace feeling mentally lighter and perhaps even subtly inspired.',
    time_spent=0.5,
    from_state_id='2c9a37e7-c684-4dd7-8a94-f04c772a4892',
    to_state_id='e93d2403-da34-4cf9-a941-22504ca44ee6'
)

...

MyOwnState(
    uuid='5dc8517d-70d5-41fa-80db-a31bc208e4b9',
    last_action='Take a Short Digestive Walk',
    previous_state_id='ea2b299d-f884-4c61-b030-81aa1986e0f3',
    state_variables=StateVariables(
        context='Edgar Allan Poe stepped outside for a short digestive walk after his light noon meal. The cold air
of the cloudy winter afternoon greeted him once again, though its sharpness seemed less biting with his improved 
state of health and energy. The reflective tone of the snow-laden streets complemented his slowly recovering senses
and creative flow, providing quiet inspiration and aiding a sense of tranquility. Walking at an unhurried pace 
allowed him to feel physically grounded while stimulating gentle circulation and aiding digestion. This secondary 
exposure to the tranquil winter landscape reinforced the mental clarity he had been cultivating throughout the day.
The brief excursion left him slightly refreshed and more attuned to the calm rhythms of the day, though physical 
exertion and the residual effects of his hangover still influenced his state.',
        pain_level=41,
        pleasure_level=67,
        health_level=64,
        focus_level=62
    ),
    environment_variables=EnvironmentVariables(
        context='Edgar Allan Poe stepped outside once again after a light noon meal, embarking on a short and 
gentle walk to aid digestion and recovery. The winter afternoon retains its serene charm, with overcast skies and 
the muffled quiet of snow-laden streets setting a contemplative, almost poetic mood. Poe is bundled warmly against 
the chill, taking care to keep his movements unhurried to avoid taxing his still-recovering body. The streets 
remain sparsely populated, maintaining a tranquil atmosphere that encourages a blend of physical grounding and 
mental calm. The walk serves as an extension of his earlier reflective outings, offering subtle yet valuable 
moments of clarity and inspiration. The air smells faintly of woodsmoke, and the subdued noises of distant 
carriages and passing pedestrians enhance the scene without intruding upon its peacefulness. By the time he returns
to his quarters, he feels better prepared to continue his day, both physically refreshed and mentally stimulated.',
        weather='cloudy',
        hour_of_day=14.0,
        day_of_week='sunday'
    )
)

Action(
    name='Warm Up and Reflect by the Fireplace',
    details='Now back in your quarters after the refreshing winter walk, take time to warm up and reflect by the 
fireplace if there is one available, or otherwise near the oil lamp for its warmth. Sit comfortably in a chair, 
perhaps with a blank notebook or a favorite piece of writing that has previously inspired you. Allow the walk’s 
gentle inspirations to settle into concrete ideas or impressions. Use this time for a quiet moment to pause and 
mentally organize before deciding on your next course of action. Ensure the atmosphere remains cozy and 
uninterrupted, as this is a key moment for rest and creativity to consolidate.',
    time_spent=0.5,
    from_state_id='5dc8517d-70d5-41fa-80db-a31bc208e4b9',
    to_state_id='14e5dcba-0f7f-4bc8-9b7e-5a995ed0e8f1'
)

MyOwnState(
    uuid='14e5dcba-0f7f-4bc8-9b7e-5a995ed0e8f1',
    last_action='Warm Up and Reflect by the Fireplace',
    previous_state_id='5dc8517d-70d5-41fa-80db-a31bc208e4b9',
    state_variables=StateVariables(
        context='After returning from a gentle digestive walk in the cold but tranquil winter afternoon, Edgar 
Allan Poe decided to spend quiet, reflective moments near a source of warmth, perhaps by a fireplace or an oil 
lamp. This serves as an opportunity to physically warm up and mentally process the inspirations from his walk. 
Sitting comfortably, he reflects on the subdued poetry of the snowy streets and the impressions from his earlier 
creative writing. This reflective pause is designed to encourage both physical recovery and creative consolidation,
offering a bridge to more structured decisions about how to spend the rest of the day.',
        pain_level=40,
        pleasure_level=70,
        health_level=66,
        focus_level=65
    ),
    environment_variables=EnvironmentVariables(
        context='After returning to his quarters from a brief and brisk winter walk, Edgar Allan Poe takes a moment
to warm himself and reflect quietly. The room, still softly lit by the oil lamp, offers a warm and inviting 
atmosphere for contemplation. Sitting in a comfortable chair near the lamp, Poe finds a thoughtful solitude to 
organize the inspirations drawn from his walk into clear ideas. The ambient light and the lingering winter chill in
the air create a balanced contrast, making the space feel serene yet focused. Outside, the overcast winter sky 
persists, infusing the quiet Baltimore afternoon with timelessness. Time has moved closer to mid-afternoon, yet the
subdued Sunday ambience remains, allowing an uninterrupted moment of mental and physical respite.',
        weather='cloudy',
        hour_of_day=14.5,
        day_of_week='sunday'
    )
)

Action(
    name='Resume Focused Writing with Enhanced Clarity',
    details="With renewed warmth and inspiration from your reflective pause near the fireplace, return to your 
writing desk to build upon the ideas that have been brewing. If you feel energized and clear-minded, use this 
session to create or polish a new section of your work, drawing from the serene winter impressions you've absorbed 
throughout the day. Focus on quality over quantity, ensuring the ideas meld smoothly and effectively. Keep the work
immersive yet measured, avoiding overexertion, and allow the deliberate pace to maximize both enjoyment and 
productivity.",
    time_spent=1.0,
    from_state_id='14e5dcba-0f7f-4bc8-9b7e-5a995ed0e8f1',
    to_state_id='2670f358-28da-47fe-8cf2-f296c908aad6'
)

MyOwnState(
    uuid='2670f358-28da-47fe-8cf2-f296c908aad6',
    last_action='Resume Focused Writing with Enhanced Clarity',
    previous_state_id='14e5dcba-0f7f-4bc8-9b7e-5a995ed0e8f1',
    state_variables=StateVariables(
        context='Having warmed up and indulged in a reflective pause by the oil lamp, Edgar Allan Poe moves back to
his desk feeling mentally refreshed and inspired. This new writing session becomes an intentional blend of 
creativity and structure, as he taps into the tranquility and clarity cultivated throughout his introspective 
Sunday. The serene winter impressions from earlier outings continue to influence and shape his compositions, while 
his slowly recovering body allows him to remain steady and focused without significant strain. Despite residual 
heaviness from the lingering hangover, his overall state now supports richer and more deliberate creative 
engagement.',
        pain_level=38,
        pleasure_level=72,
        health_level=68,
        focus_level=69
    ),
    environment_variables=EnvironmentVariables(
        context='As Edgar Allan Poe transitions from quiet reflection near a source of warmth back to his writing 
desk, the serene and contemplative day fosters a rich environment for the depth and clarity required for focused 
creative work. Having spent time processing inspiration from the tranquil winter landscape and his earlier walk, 
Edgar now immerses himself in the task of expanding upon or refining his writing. The overcast weather outside 
continues to offer a steady visual reminder of the muted beauty of winter, while the warm light of the oil lamp 
enhances the sense of a private creative cocoon. The peace of mid-afternoon Sunday in Baltimore persists, lending 
itself to slow, uninterrupted thought. This session is poised to be thoughtful and productive, with attention paid 
to the fusion of structured ideas and creative freedom.',
        weather='cloudy',
        hour_of_day=15.5,
        day_of_week='sunday'
    )
)

In [256]:
actions = life.sample_actions()
for action in actions:
    rprint(action)

Action(
    name='Resume Writing with Renewed Focus',
    details='Having taken a restorative break, return to your desk to channel the ideas that have percolated during
stillness into a renewed session of creative writing. Focus now on refining or expanding the earlier work if it 
feels promising, or begin exploring new concepts inspired by the walk or the serene winter setting. Use this time 
to balance structure and flow, aiming for a more intentional creative output than in the initial unstructured 
session. The activity should remain immersive yet manageable in intensity, keeping in mind your gradual recovery 
from the hangover.',
    time_spent=1.0,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Enjoy a Light Noon Meal',
    details='Pause your writing to nourish yourself with a light, simple meal. This will help sustain your energy 
as you continue to recover from lingering hangover symptoms and provide a sense of equilibrium to your ongoing 
creative efforts. Consider having something warm and soothing, such as broth or bread, to complement the cold and 
overcast winter atmosphere. The act of eating not only addresses physical needs but also offers a momentary mindful
break, allowing your thoughts to organically expand or settle before your next creative phase.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Take a Short Digestive Walk',
    details='After your light noon meal, step outside again for a brief, refreshing walk to aid digestion and 
maintain the low pace of recovery. The crisp, cloudy winter afternoon continues to offer its serene beauty and 
calm, which can inspire further reflection or creativity. Bundle up adequately given the temperature, and keep the 
walk gentle and unhurried to avoid overexertion as you continue your gradual recovery from the hangover. Use this 
moment to feel physically grounded and mentally uncluttered, returning to your quarters both refreshed and ready 
for continued creative exploration.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Warm Up and Reflect by the Fireplace',
    details='Now back in your quarters after the refreshing winter walk, take time to warm up and reflect by the 
fireplace if there is one available, or otherwise near the oil lamp for its warmth. Sit comfortably in a chair, 
perhaps with a blank notebook or a favorite piece of writing that has previously inspired you. Allow the walk’s 
gentle inspirations to settle into concrete ideas or impressions. Use this time for a quiet moment to pause and 
mentally organize before deciding on your next course of action. Ensure the atmosphere remains cozy and 
uninterrupted, as this is a key moment for rest and creativity to consolidate.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Resume Focused Writing with Enhanced Clarity',
    details="With renewed warmth and inspiration from your reflective pause near the fireplace, return to your 
writing desk to build upon the ideas that have been brewing. If you feel energized and clear-minded, use this 
session to create or polish a new section of your work, drawing from the serene winter impressions you've absorbed 
throughout the day. Focus on quality over quantity, ensuring the ideas meld smoothly and effectively. Keep the work
immersive yet measured, avoiding overexertion, and allow the deliberate pace to maximize both enjoyment and 
productivity.",
    time_spent=1.0,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Organize Writing Notes for Later Review',
    details="After finishing a focused writing session, dedicate some time to organizing your notes into coherent 
categories for easier future reference. This step will not only help solidify the day's creative output but also 
allow for better flow in subsequent sessions. Use index cards, a journal, or any preferred system to ensure ideas 
are accessible. Such organization will create a smoother transition into tomorrow's tasks while lending a sense of 
accomplishment to conclude the current session.",
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Prepare a Soothing Evening Tea',
    details="After a day's work of writing and reflection, wind down by preparing and enjoying a soothing cup of 
tea. Choose a calming blend like chamomile or lavender to complement the relaxing ambiance of the late afternoon. 
Sip slowly and allow this time to signify a shift from creative focus to personal relaxation. The herbal tea also 
aids in gentle cleansing and promoting better physical recovery as the lingering effects of the hangover continue 
to diminish.",
    time_spent=0.3,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Evening Reflection and Journaling',
    details="Set aside some time in the evening to journal about the day’s impressions and inspirations. Use this 
activity to document the highlights of your creative efforts, your reflections from the serene walks, and any 
lingering ideas that can serve as seeds for future work. Maintain a relaxed and open tone, allowing this moment to 
blend creativity with self-reflection. This practice will also help you unwind and mentally close the loop on 
today's themes.",
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Settle Down with a Good Book',
    details='Transition into a fully restful evening by reading a favored piece of literature. Choose something you
find enriching or calming to enhance the tranquil mood of the day’s finale. This act not only nurtures the creative
mind but also provides an immersive escape, further assisting your recovery. Ensure a comfortable seating 
arrangement near the light of the oil lamp for optimal relaxation.',
    time_spent=1.0,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Prepare for Rest with Evening Meditation',
    details='Conclude the day with a brief meditation session to fully relax the body and calm the mind. Focus on 
your breathing and let go of any residual strain or tension. Incorporate light stretching if needed, and reflect on
the sense of gratitude for the quiet, contemplative day. This step will ensure you are grounded, fully relaxed, and
prepared for a restful night’s sleep.',
    time_spent=0.3,
    from_state_id=None,
    to_state_id=None
)

In [258]:
import pandas

pandas.options.plotting.backend = "plotly"

df = pandas.DataFrame(
    [
        {**x.state_variables.model_dump(), **x.environment_variables.model_dump()}
        for x in life.states
    ]
)[
    [
        "pain_level",
        "pleasure_level",
        "health_level",
        "focus_level",
        "hour_of_day",
    ]
]

In [2]:
df.set_index("hour_of_day").plot()

NameError: name 'df' is not defined